### Load Data

#### Connect to Spark & Elasticsearch, gather raw data

In [1]:
import sys
!{sys.executable} -m pip install findspark plotly colorlover scipy numpy

    100% |████████████████████████████████| 38.4MB 1.0MB/s ta 0:00:011    45% |██████████████▋                 | 17.6MB 18.0MB/s eta 0:00:02    69% |██████████████████████▍         | 26.8MB 29.3MB/s eta 0:00:01    91% |█████████████████████████████▎  | 35.1MB 11.2MB/s eta 0:00:01    94% |██████████████████████████████▍ | 36.4MB 5.6MB/s eta 0:00:01
  Running setup.py bdist_wheel for retrying ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/d7/a9/33/acc7b709e2a35caa7d4cae442f6fe6fbf2c43f80823d46460c
Successfully built retrying


In [2]:
import findspark
import os

from pyspark import SparkContext
from pyspark.sql import SparkSession, SQLContext

os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars jars/elasticsearch-spark-20_2.11-6.5.1.jar pyspark-shell'

findspark.init()

sc = SparkContext(appName="esAnalytics")
sqlContext = SQLContext(sc)

spark = SparkSession.builder \
    .master("local[*]") \
    .appName("meetup") \
    .config("spark.some.config.option", "some-value") \
    .config("spark.sql.crossJoin.enabled", "true") \
    .getOrCreate()

# spark = SparkSession.builder \
#     .master("yarn") \
#     .appName("meetup") \
#     .config("spark.some.config.option", "some-value") \
#     .config("spark.sql.crossJoin.enabled", "true") \
#     .getOrCreate()

In [3]:
# from json import loads, dumps

# es_read_conf = {
# "es.nodes" : 'elastic',
# "es.port" : '9200',
# "es.resource" : 'meetup-rawdata-*/default'
# }

# raw_data = sc.newAPIHadoopRDD(
# inputFormatClass="org.elasticsearch.hadoop.mr.EsInputFormat",
# keyClass="org.apache.hadoop.io.NullWritable",
# valueClass="org.elasticsearch.hadoop.mr.LinkedMapWritable",
# conf=es_read_conf)

# raw_data = raw_data.map(lambda v: loads(dumps(v[1])))

# df = sqlContext.createDataFrame(raw_data)

In [4]:
df = spark.read.json("./data/meetup-rawdata/*json")
# add - select only columns used later on
df.cache()
df.count()

985410

#### Select only last response for each rsvp_id

In [5]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, col

rsvpWindowSpec = Window.partitionBy(df["rsvp_id"]).orderBy(df["mtime"].desc())

df = df \
    .withColumn("rowId", row_number().over(rsvpWindowSpec)) \
    .where("rowId = 1") \
    .sort("rsvp_id")

#### Establish timezone, day_of_week_local, hour_local, minute_local of event.event_time based on venue.venue_geo

In [6]:
# import pytz

# from datetime import datetime
# from tzwhere import tzwhere

# from pyspark.sql.functions import udf
# from pyspark.sql.types import StringType, ArrayType

# @udf(StringType())
# def udf_timezone_by_geo(lat, lon):
#     t = tzwhere.tzwhere()
    
#     return t.tzNameAt(float(lat), float(lon))

# @udf(ArrayType(StringType()))
# def udf_localize_with_timezone(utc_time, timezone_str):
    
#     epoch_utc = int(utc_time)
    
#     timezone_str = timezone_str.strip()
    
#     # check if provided in ms or s:
#     if len(str(epoch_utc)) == 13:
#         epoch_utc = epoch_utc / 1000

#     # get time in UTC
#     utc_dt = datetime.utcfromtimestamp(epoch_utc)

#     # convert it to tz
#     tz = pytz.timezone(timezone_str)
#     dt = utc_dt.astimezone(tz)

#     offset = dt.utcoffset().total_seconds()

#     local_dt = datetime.utcfromtimestamp(epoch_utc + offset)
    
#     parts = dict(year_local=local_dt.year,
#                  month_local=local_dt.month, 
#                  day_local=local_dt.day, 
#                  weekday_local=local_dt.isoweekday(),
#                  hour_local=local_dt.hour, 
#                  minute_local=local_dt.minute)
    
#     return list(parts.values())

In [7]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, ArrayType

from datetime import datetime


@udf(ArrayType(StringType()))
def udf_extract_date_parts(utc_time):
    epoch_utc = int(utc_time)
    
    # check if provided in ms or s:
    if len(str(epoch_utc)) == 13:
        epoch_utc = epoch_utc / 1000

    # get time in UTC
    utc_dt = datetime.utcfromtimestamp(epoch_utc)
    
    parts = dict(year_local=utc_dt.year,
                 month_local=utc_dt.month, 
                 day_local=utc_dt.day, 
                 weekday_local=utc_dt.isoweekday(),
                 hour_local=utc_dt.hour, 
                 minute_local=utc_dt.minute)
    
    return list(parts.values())

In [8]:
# # to optimize matching event.time with venue.lat/venue.lon create dict with distinct venues

# from pyspark.sql.functions import concat, lit

# venueGeoDict = df \
#     .select(col("venue.lat"), col("venue.lon")) \
#     .distinct() \
#     .withColumn("key", concat(col("lat"), lit("_"), col("lon"))) \
#     .withColumn("event_timezone", udf_timezone_by_geo(col("lat"), col("lon"))) \
#     .select(col("key"), col("event_timezone"))

# venueGeoDict.show(5)

In [9]:
# dfWithEventTimezone = df \
#     .join(venueGeoDict, concat(col("venue.lat"), lit("_"), col("venue.lon")) == venueGeoDict.key, 'cross') 

dfWithEventTimeParts = df \
    .where(col("event.time").isNotNull()) \
    .withColumn("event_time_parts", udf_extract_date_parts(df.event.time))

dfWithEventTimeParts.cache()

DataFrame[@timestamp: string, event: struct<event_id:string,event_name:string,event_url:string,time:bigint>, group: struct<group_city:string,group_country:string,group_geo:string,group_id:bigint,group_lat:double,group_lon:double,group_name:string,group_state:string,group_topics:array<string>,group_urlname:string>, guests: bigint, member: struct<member_id:bigint,member_name:string,other_services:struct<facebook:struct<identifier:string>,flickr:struct<identifier:string>,linkedin:struct<identifier:string>,tumblr:struct<identifier:string>,twitter:struct<identifier:string>>,photo:string>, mtime: bigint, response: string, rsvp_id: bigint, venue: struct<lat:double,lon:double,venue_geo:string,venue_id:bigint,venue_name:string>, visibility: string, rowId: int, event_time_parts: array<string>]

In [10]:
dfWithEventTimeParts \
    .select(col("event.time"), col("event_time_parts")) \
    .show(5)

+-------------+--------------------+
|         time|    event_time_parts|
+-------------+--------------------+
|1546268400000|[2018, 12, 31, 1,...|
|1575574200000|[2019, 12, 5, 4, ...|
|1575574200000|[2019, 12, 5, 4, ...|
|1575574200000|[2019, 12, 5, 4, ...|
|1575574200000|[2019, 12, 5, 4, ...|
+-------------+--------------------+
only showing top 5 rows



### Analyze

#### Calculate & visualize most distinguishable distributions of meetings in particular day_of_week_local by tag (Jensen–Shannon divergence)

In [11]:
# total distribution

from pyspark.sql.functions import lit, count, udf, collect_list
from pyspark.sql.types import StringType, DoubleType, MapType

countByAllWindowSpec = Window.partitionBy(lit(1))

totalWeekdayDistribution = dfWithEventTimeParts \
    .withColumn("event_isoweekday", dfWithEventTimeParts.event_time_parts[3]) \
    .select(col("rsvp_id"), col("event_isoweekday"), count(col("rsvp_id")).over(countByAllWindowSpec).alias("count_all")) \
    .groupBy(col("event_isoweekday"), col("count_all")) \
    .count() \
    .sort("event_isoweekday") \
    .withColumn("weekday_total_dist", col("count")/col("count_all")) \
    .groupBy() \
    .agg(collect_list(col("weekday_total_dist")).alias("weekday_total_dist"))

totalWeekdayDistribution.cache()

DataFrame[weekday_total_dist: array<double>]

In [12]:
totalWeekdayDistribution.collect()

[Row(weekday_total_dist=[0.07185578359124661, 0.13864492108344612, 0.16100379332347023, 0.15654295710950153, 0.12406201137295872, 0.20754597914723844, 0.14034455437213836])]

In [13]:
# distribution by group topic
from pyspark.sql.functions import explode, lower, coalesce, abs, max, rank
from pyspark.sql.types import Row

countByTopicWindowSpec = Window.partitionBy("group_topic")

topicWeekdayDistributionTmp = dfWithEventTimeParts \
    .withColumn("event_isoweekday", dfWithEventTimeParts.event_time_parts[3]) \
    .select(col("rsvp_id"), col("event_isoweekday"), explode(col("group.group_topics")).alias("group_topic_map")) \
    .withColumn("group_topic", col("group_topic_map")) \
    .withColumn("total_topic_count", count("rsvp_id").over(countByTopicWindowSpec)) \
    .drop("group_topic_map") \
    .groupBy(col("event_isoweekday"), col("group_topic"), col("total_topic_count")) \
    .count() \
    .sort(col("group_topic"), col("event_isoweekday"))

# topicWeekdayDistributionTmp.show(30)

topics = topicWeekdayDistributionTmp.select(col("group_topic").alias("group_topic_tmp")).distinct()
weekdays = sc.parallelize(list(range(7))).map(lambda x: Row(event_isoweekday_tmp=str(1 + int(x)))).toDF()
cross = weekdays.crossJoin(topics).withColumn("count_tmp", lit(0))

# ensure that every topic has entry for every weekday (even if no meetings took place on that weekday)
topicWeekdayDistribution = cross \
    .join(topicWeekdayDistributionTmp, (topicWeekdayDistributionTmp.event_isoweekday == cross.event_isoweekday_tmp) & (topicWeekdayDistributionTmp.group_topic == cross.group_topic_tmp), how='outer') \
    .withColumn("event_isoweekday", col("event_isoweekday_tmp")) \
    .withColumn("group_topic", col("group_topic_tmp")) \
    .withColumn("count", coalesce("count", "count_tmp")) \
    .withColumn("total_topic_count", coalesce("total_topic_count", lit(-1))) \
    .drop("event_isoweekday_tmp", "group_topic_tmp", "count_tmp") \
    .sort("group_topic", "event_isoweekday") \
    .withColumn("weekday_topic_dist", abs(col("count")/col("total_topic_count"))) \
    .groupBy("group_topic") \
    .agg(collect_list(col("weekday_topic_dist")).alias("weekday_topic_dist"), max(col("total_topic_count")).alias("topic_count")) \
    .withColumn("topic_count_rank", rank().over(Window.partitionBy(lit(1)).orderBy(col("topic_count"))))

topicWeekdayDistribution.show(20)
topicWeekdayDistribution.cache()

+--------------------+--------------------+-----------+----------------+
|         group_topic|  weekday_topic_dist|topic_count|topic_count_rank|
+--------------------+--------------------+-----------+----------------+
|Art classes for h...|[0.0, 0.0, 0.0, 1...|          1|               1|
|Audience Development|[0.0, 0.0, 0.0, 0...|          1|               1|
|Colorado Real Estate|[0.0, 0.0, 0.0, 0...|          1|               1|
|Paddle Sports on ...|[0.0, 0.0, 0.0, 0...|          1|               1|
|      Primal Fitness|[0.0, 0.0, 1.0, 0...|          1|               1|
|   Real Life Stories|[1.0, 0.0, 0.0, 0...|          1|               1|
|    Resource Sharing|[0.0, 0.0, 0.0, 0...|          1|               1|
|          Stillbirth|[0.0, 0.0, 1.0, 0...|          1|               1|
|       Underemployed|[1.0, 0.0, 0.0, 0...|          1|               1|
|Young Married Cat...|[0.0, 0.0, 1.0, 0...|          1|               1|
|        makeup party|[0.0, 0.0, 0.0, 0...|        

DataFrame[group_topic: string, weekday_topic_dist: array<double>, topic_count: bigint, topic_count_rank: int]

In [14]:
# JS Divergence UDF

from numpy import asarray, e
from scipy import stats

from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType

@udf(DoubleType())
def udf_jsd(p, q, base=e):
    '''
        Implementation of pairwise `jsd` based on  
        https://en.wikipedia.org/wiki/Jensen%E2%80%93Shannon_divergence
    '''
    try:
        ## convert to np.array
        p, q = asarray(p), asarray(q)
        ## normalize p, q to probabilities
        p, q = p/p.sum(), q/q.sum()

        m = 1./2*(p + q)

        return float(stats.entropy(p,m, base=base)/2. +  stats.entropy(q, m, base=base)/2.)
    except:
        return -1

In [15]:
# calculate Jensen-Shannon Divergence per topic & select 10 highest
jsDivergence = topicWeekdayDistribution \
    .crossJoin(totalWeekdayDistribution) \
    .withColumn("jsd", udf_jsd(col("weekday_topic_dist"), col("weekday_total_dist"))) \
    .sort(col("jsd").desc()) \
    .where(topicWeekdayDistribution.topic_count > 1000)

jsDivergence.cache()

DataFrame[group_topic: string, weekday_topic_dist: array<double>, topic_count: bigint, topic_count_rank: int, weekday_total_dist: array<double>, jsd: double]

In [16]:
jsDivergence.sort(col("jsd").desc()).show(100)

+--------------------+--------------------+-----------+----------------+--------------------+-------------------+
|         group_topic|  weekday_topic_dist|topic_count|topic_count_rank|  weekday_total_dist|                jsd|
+--------------------+--------------------+-----------+----------------+--------------------+-------------------+
|             Collage|[0.00753768844221...|       1194|           32265|[0.07185578359124...|0.17897320811308526|
|                APIs|[0.19287833827893...|       1011|           32154|[0.07185578359124...|0.16909681279761324|
|    Java Concurrency|[0.02230843840931...|       1031|           32166|[0.07185578359124...|0.16108217753068874|
|Mobile Product Ma...|[0.03656998738965...|       1586|           32396|[0.07185578359124...| 0.1419759906808842|
|Load Testing and ...|[0.02977905859750...|       1041|           32178|[0.07185578359124...| 0.1328734581206049|
|        ASP.NET Core|[0.21396993810786...|       1131|           32235|[0.0718557835912

In [17]:
def plot_hist(data_list):
    import plotly.graph_objs as go
    from plotly.offline import init_notebook_mode, iplot
    from math import ceil
    from plotly import tools
    
    init_notebook_mode(connected=True)
    
    from json import loads
    import colorlover as cl

    no = len(data_list)
    
    colsNo = 5
    rowsNo = ceil(no/colsNo)
    
    color = cl.scales['11']['div']['RdBu']
    
    select = [1,2,-1,-2,3]
    color = [color[i] for i in select]
    
    fig = tools.make_subplots(rows=rowsNo, 
                              cols=colsNo, 
                              subplot_titles=['<b>{}</b><br>(JSD: {:.2f})'.format(loads(x).get('group_topic', ''),
                                                                    loads(x).get('jsd', '')) for x in data_list],
                              
                              shared_yaxes=True
                             )

    fig['layout'].update(height=rowsNo*300, width=colsNo*300, title='Most characteristic weekday dist', showlegend=False)

    i = 0
    
    rows = [x+1 for x in range(rowsNo)]
    cols = [x+1 for x in range(colsNo)]
    
    combos = [(i,j) for i in rows for j in cols]
    
    ci = 0
    
    for data in data_list:
        ci += 1

        c = color[(ci % colsNo)]
        combo = combos[i]
        
        cur_row = combo[0]
        cur_col = combo[1]
        
        data = loads(data)

        x = [x+1 for x in range(7)]
        y1 = data.get('weekday_topic_dist', [0 for x in range(6)])
        y2 = data.get('weekday_total_dist', [0 for x in range(6)])

        title = data.get('group_topic', 'na')

        fig.append_trace(go.Bar(x=x,y=y1,marker=dict(color=[c for i in range(7)])), cur_row, cur_col)
        fig.append_trace(go.Scatter(x=x,y=y2,mode='lines',line=dict(color='gray',width=3,shape='spline')), cur_row, cur_col)
        
        i += 1
        
    iplot(fig, filename='make-subplots-multiple-with-titles')
    
entries = jsDivergence.toJSON().take(200)

plot_hist(entries)

This is the format of your plot grid:
[ (1,1) x1,y1 ]      [ (1,2) x2,y1 ]      [ (1,3) x3,y1 ]      [ (1,4) x4,y1 ]      [ (1,5) x5,y1 ]    
[ (2,1) x6,y2 ]      [ (2,2) x7,y2 ]      [ (2,3) x8,y2 ]      [ (2,4) x9,y2 ]      [ (2,5) x10,y2 ]   
[ (3,1) x11,y3 ]     [ (3,2) x12,y3 ]     [ (3,3) x13,y3 ]     [ (3,4) x14,y3 ]     [ (3,5) x15,y3 ]   
[ (4,1) x16,y4 ]     [ (4,2) x17,y4 ]     [ (4,3) x18,y4 ]     [ (4,4) x19,y4 ]     [ (4,5) x20,y4 ]   
[ (5,1) x21,y5 ]     [ (5,2) x22,y5 ]     [ (5,3) x23,y5 ]     [ (5,4) x24,y5 ]     [ (5,5) x25,y5 ]   
[ (6,1) x26,y6 ]     [ (6,2) x27,y6 ]     [ (6,3) x28,y6 ]     [ (6,4) x29,y6 ]     [ (6,5) x30,y6 ]   
[ (7,1) x31,y7 ]     [ (7,2) x32,y7 ]     [ (7,3) x33,y7 ]     [ (7,4) x34,y7 ]     [ (7,5) x35,y7 ]   
[ (8,1) x36,y8 ]     [ (8,2) x37,y8 ]     [ (8,3) x38,y8 ]     [ (8,4) x39,y8 ]     [ (8,5) x40,y8 ]   
[ (9,1) x41,y9 ]     [ (9,2) x42,y9 ]     [ (9,3) x43,y9 ]     [ (9,4) x44,y9 ]     [ (9,5) x45,y9 ]   
[ (10,1) x46,y10 ]   [ (10

#### Calculate 'New Years Resolutions Effect' to establish which tags gained most interest inbetween december/january

In [18]:
# all available topics (if occured in one month and not other)
t = df \
    .select(explode(col("group.group_topics")).alias("topic")) \
    .select(col("topic").getItem("urlkey").alias("topic")) \
    .distinct()

# select topics from january, 2019
m1 = dfWithEventTimeParts \
    .where((col("event_time_parts")[0] == '2018') & (col("event_time_parts")[2] == '5')) \
    .select(explode("group.group_topics").alias("topic")) \
    .withColumn("topic_m1", col("topic").getItem("urlkey")) \
    .groupBy("topic_m1") \
    .count() \
    .alias("m1")

# select topics from december, 2018
m2 = dfWithEventTimeParts \
    .where((col("event_time_parts")[0] == '2018') & (col("event_time_parts")[2] == '4')) \
    .select(explode("group.group_topics").alias("topic")) \
    .withColumn("topic_m2", col("topic").getItem("urlkey")) \
    .groupBy("topic_m2") \
    .count() \
    .alias("m2")

# calculate increase in interest per topic & select 10 highest
increase = t \
    .join(m2, t.topic == m2.topic_m2, how='full') \
    .join(m1, t.topic == m1.topic_m1, how='full') \
    .withColumn("m1", coalesce(col("m1.count"), lit("0"))) \
    .withColumn("m2", coalesce(col("m2.count"), lit("0"))) \
    .select(col("topic"), col("m1"), col("m2")) \
    .withColumn("change", ((col("m1")+col("m2"))/2)*(col("m1")/col("m2"))) \
    .sort(col("change").desc())

increase.cache()

increase.limit(20).show()

# show topics that were absent in december, 2018 but appeared in january, 2019
new_topics = increase \
    .where((col("m1") > 0) & (col("m2") == 0))

new_topics.show()

AnalysisException: "Can't extract value from topic#6387: need struct type but got string;"